## Fixed Effect Model (i really pray it works)

In [1]:
import pandas as pd
import numpy as np
from linearmodels.panel import PanelOLS

In [12]:
#import data first
training_data=pd.read_csv('../data/final/training_model_data.csv',header=0)
test_data=pd.read_csv('../data/final/incompl_test_model_data.csv',header=0)
test_data=test_data[training_data.columns]
combined=pd.concat([training_data,test_data],axis=0)

In [13]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))
combined['tradeagreementindex']=scaler.fit_transform(combined['tradeagreementindex'].values.reshape(-1,1))

In [ ]:
combined['composite_d']=(-1)*0.5*combined['tradeagreementindex']+(-1)*0.5*combined['sentiment_index']
combined=combined.drop(columns=['sentiment_index','tradeagreementindex'],axis=1)

In [33]:
df = combined.melt(
    id_vars=['year','country_a','country_b','composite_d'],
    value_vars=['bec_1','bec_2','bec_3','bec_4','bec_5','bec_6','bec_7','bec_8'],
    var_name='sector',
    value_name='value'
)
df=df.rename(columns={'composite_d':'D'})

In [34]:
df = df[(df['country_a'] != 'ARE') & (df['country_b'] != 'ARE')]


In [31]:
df[df['value']==0]

,year,country_a,country_b,D,sector,value


In [ ]:
# df=df.sort_values(['country_a','country_b','sector','year'])
# df['log_export']=np.log(df['value'])
# df['delta_log_export']=df.groupby(['country_a', 'country_b', 'sector'])['log_export'].diff()
# df['entity_id'] = df['country_a'] + '_' + df['country_b'] + '_' + df['sector']

In [23]:
df['delta_log_export'].isna().sum()

2686

In [35]:
df=df.sort_values(['country_a','country_b','sector','year'])
df['log_export']=np.log(df['value'])
df['delta_log_export']=df.groupby(['country_a', 'country_b', 'sector'])['log_export'].diff()
df['entity_id'] = df['country_a'] + '_' + df['country_b'] + '_' + df['sector']

# Now create the proper 2-level MultiIndex for PanelOLS

panel_data = df.set_index(['entity_id', 'year'])

# # # Now run the model
formula = 'delta_log_export ~ D + EntityEffects + TimeEffects'
model = PanelOLS.from_formula(formula, panel_data)
results = model.fit(cov_type='clustered', cluster_entity=True)
print(results)
# all_years = range(df['year'].min(), df['year'].max() + 1)
# all_entities = df['entity_id'].unique()
# full_index = pd.MultiIndex.from_product(
#     [all_entities, all_years],
#     names=['entity_id', 'year']
# )

# # Reindex to include all years, filling missing rows with NaN
# panel_data = panel_data.reindex(full_index)

c:\Users\rob-l\Documents\NUS\Y3S2\DSE3101\T5G1\venv\Lib\site-packages\linearmodels\panel\model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


                          PanelOLS Estimation Summary                           
Dep. Variable:       delta_log_export   R-squared:                        0.0003
Estimator:                   PanelOLS   R-squared (Between):             -0.0738
No. Observations:               36992   R-squared (Within):              -0.0035
Date:                Tue, Apr 08 2025   R-squared (Overall):             -0.0073
Time:                        07:55:57   Log-likelihood                -1.695e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      8.9201
Entities:                        2176   P-value                           0.0028
Avg Obs:                       17.000   Distribution:                 F(1,34799)
Min Obs:                       17.000                                           
Max Obs:                       17.000   F-statistic (robust):             7.9452
                            

In [ ]:
from linearmodels import PanelOLS

formula = 'delta_log_export ~ D + EntityEffects + TimeEffects'
model = PanelOLS.from_formula(formula, panel_data)
results = model.fit(cov_type='clustered', cluster_entity=True)

c:\Users\rob-l\Documents\NUS\Y3S2\DSE3101\T5G1\venv\Lib\site-packages\linearmodels\panel\model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


ValueError: array must not contain infs or NaNs